<div align="center">
 <img src="https://raw.githubusercontent.com/matheusmota/dataviz2018/master/resources/images/logo_facens_pos.png" width="150px">
 <h1> Dataviz - Data Science Specialization Program - FACENS</h1>
</div>
<br><br>
# Exercício 1 - Primeiro contato com o Kaggle
(valendo nota)

* **Data de entrega:** até o final da aula 
* **Professor:**  Matheus Mota
* **Aluno:** Danilo Duarte
* **RA:** 203085

## Questão 1
**Enunciado:**  Este notebook está associado ao *Kaggle Dataset* chamado "Exercício 1". Este *Kaggle Dataset* possui dois arquivos em formato CSV (anv.csv e BR_eleitorado_2016_municipio ). Escolha um dos datasets disponíveis e já conhecidos, a seu critério. Uma vez definido o csv, escolha no mínimo 7 e no máximo 12 variáveis (colunas) que você avalia como sendo relevantes. Para cada uma das suas variáveis escolhidas, forneça:


### Questão 1 - Item A - Classificação das variáveis

Classifique todas as variáveis escolhidas, e construa um dataframe com sua resposta.

In [ ]:
import pandas as pd
df = pd.read_csv('../input/dataviz-facens-20182-aula-1-exerccio-2/anv.csv', delimiter=',')
df = df.drop(['codigo_ocorrencia','aeronave_matricula', 'aeronave_modelo','aeronave_pais_registro','aeronave_fase_operacao_icao','aeronave_fase_operacao','aeronave_operador_categoria','aeronave_modelo','aeronave_tipo_icao',
         'aeronave_motor_tipo','aeronave_motor_quantidade','aeronave_pmd','aeronave_pmd_categoria',
        'aeronave_pais_fabricante','aeronave_pais_registro','aeronave_registro_categoria',
        'aeronave_registro_segmento','aeronave_fase_operacao','aeronave_fase_operacao_icao','aeronave_tipo_operacao',
        'aeronave_nivel_dano','aeronave_dia_extracao'], axis=1)
df = df.dropna()
df.head(1)

Verificando a existencia de valores nulos após limpeza:

In [ ]:
df.isnull().values.sum()

Realizando a classificação das variáveis:

In [ ]:
tab_classific = pd.concat([pd.DataFrame([_], columns=['Variavel']) for _ in df.columns], ignore_index=True)
tab_classific['Classificação'] = ['Qualitativa Nominal','Qualitativa Nominal','Quantitativa Discreta',
                                  'Quantitativa Discreta','Qualitativa Nominal','Qualitativa Nominal',
                                  'Quantitativa Discreta']
tab_classific.head(8)

### Questão 1 - Item B - Tabela de frequência

Construa uma tabela de frequência para cada uma das **variáveis qualitativas** que você escolheu (caso não tenha escolhido nenhuma, deixe esta questão em branco). Uma dica: a função *value_counts()* do Pandas pode ser muito útil. =)


In [ ]:
variavel = []
counts = []
for _ in df.columns:
    variavel.append(_)
    counts.append(df[_].value_counts().sum())

frequencia = list(zip(variavel,counts))
frequencia = pd.DataFrame(frequencia, columns=['Variavel','frequencia'])
frequencia.head(8)

### Questão 1 - Item C - Representação Gráfica
Para cada uma das variáveis, produza um ou mais gráficos, usando matplotlib, que descreva seu comportamento / caracteristica. Lembre-se que estes gráficos precisam ser compatíveis com a classificação da variável. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
df.describe()

In [ ]:
df_fat = df[df['total_fatalidades'] > 0]

dfG = df_fat.groupby(['aeronave_assentos'],as_index = False).sum().sort_values(by='total_fatalidades', 
                                                                         ascending = False)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.grid(zorder=0)
ax.set_ylabel('Número de Assentos')
ax.set_xlabel('Fatalidades')
ax.bar(dfG.aeronave_assentos,dfG.total_fatalidades, color=sns.color_palette("rocket"), align='center',zorder=3)
plt.title('Fatalidade por número de Assentos')
plt.show()

Verificando os valores dos voos de destino que não possuem descrição:

In [ ]:
df_fat[df_fat['aeronave_voo_destino'] == '****'].count()

In [ ]:
df_fat[df_fat['aeronave_voo_destino'] == '###!'].count()

In [ ]:
dfD = df_fat[~df_fat['aeronave_voo_destino'].isin(['****','###!'])]
dfD = dfD[dfD['total_fatalidades'] >= 8]
dfD = dfD.groupby(['aeronave_voo_destino'],as_index = False).sum().sort_values(by='total_fatalidades', 
                                                                         ascending = False)

explode = (0.2, 0, 0, 0,0,0) 
fig1, ax1 = plt.subplots()
ax1.pie(dfD.total_fatalidades, explode=explode ,labels=dfD.aeronave_voo_destino, autopct='%1.1f%%',
        shadow=True, startangle=80)
ax1.axis('equal') 
plt.title('Destino por Fatalidade')
plt.show()

In [ ]:
x1 = df['total_fatalidades']
x2 = dfD['total_fatalidades']
plt.title('Distribuição do total de fatalidades dos dados Brutos Vs Dados limpos')
plt.boxplot ([x1, x2])
plt.show()

In [ ]:
df_ano = df_fat[df_fat['aeronave_ano_fabricacao'] > 1970]
df_ano = df_ano.groupby(['aeronave_ano_fabricacao'],as_index = False).sum().sort_values(by='total_fatalidades', 
                                                                         ascending = False)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.grid(zorder=0)
ax.set_ylabel('Número de Assentos')
ax.set_xlabel('Fatalidades')
ax.barh(df_ano.aeronave_ano_fabricacao,df_ano.total_fatalidades, align='center')
plt.title('Fatalidade por Ano de fábricação da Aeronave')
plt.show()

In [ ]:
df_percent = df.groupby(["aeronave_tipo_veiculo"], as_index= False).count().sort_values('total_fatalidades', 
                                                                           ascending=False)[['aeronave_tipo_veiculo','total_fatalidades']]
df_tota_fat = df.groupby(["aeronave_tipo_veiculo"]).sum()["total_fatalidades"]
df_percent = df_percent.merge(df_tota_fat.to_frame(),on='aeronave_tipo_veiculo',how='inner')
df_percent = df_percent.drop(5)
df_percent['percent'] = round((df_percent.total_fatalidades_y / df_percent.total_fatalidades_x) * 100, 2)
df_percent.sort_values(by='percent', ascending=False)
df_percent = df_percent[df_percent['percent'] > 3]

labels = df_percent.aeronave_tipo_veiculo
sizes = df_percent.percent
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#97AF9F']
explode = (0.05,0.05,0.2,0.05,0.05)
 
plt.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax1.axis('equal')  
plt.title('Percentual de Acidentes Fatais por\n' + 'tipo de Avião',bbox={'facecolor':'0.8', 'pad':5})
plt.tight_layout()
plt.show()